Get username dan password untuk login ke instagram

In [1]:
import json
import getpass

def getdata():
    data = {}
    data['username'] = input("Masukkan username: ")
    # input password secara tersembunyi dan balik 
    reversed_password = getpass.getpass("Masukkan password: ")[::-1]
    data['password'] = reversed_password

    # write data to json file
    with open('data.json', 'w') as outfile:
        json.dump(data, outfile)

getdata()

Membuka Chrome menggunakan webdriver

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import json
import time

In [10]:
chrome_options = Options()

chrome_options.add_argument("--headless")
# argumen to disable notification
chrome_options.add_argument("--disable-notifications")
# argumen to disable settings infobar
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--incognito")
# window size maximize
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/")

Login to instagram

In [11]:
time.sleep(3)

# take data from json file
with open('data.json') as json_file:
    data = json.load(json_file)
    data_username = data['username']
    data_password = data['password']

    unreversed_password = data_password[::-1]


unamebox = driver.find_element(By.NAME, "username")
unamebox.send_keys(data_username)

pwbox = driver.find_element(By.NAME, "password")
pwbox.send_keys(unreversed_password)

pwbox.send_keys(Keys.ENTER)

In [12]:
time.sleep(5)
driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div/div/div/div").click()
time.sleep(3)
driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/button[2]").click()

In [13]:
driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/span/div/a/div").click()
time.sleep(3)
searchbox = driver.find_element(By.CLASS_NAME, "_aauy")
time.sleep(5)

searchbox.send_keys("ittelkom")

time.sleep(3)
driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div[2]/div/div[1]/a").click()

Scrapping comment

In [14]:
time.sleep(5)

post_list = [
    "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[3]/article/div[1]/div/div[1]/div[1]/a",
    "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[3]/article/div[1]/div/div[1]/div[2]/a",
    "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[3]/article/div[1]/div/div[1]/div[3]/a",
    "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[3]/article/div[1]/div/div[2]/div[1]/a",
    "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[3]/article/div[1]/div/div[2]/div[2]/a",
    "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[3]/article/div[1]/div/div[2]/div[3]/a",
]

# scrap comment dari post_list
for post in post_list:
    driver.find_element(By.XPATH, post).click()
    time.sleep(3)
    blok_komentar = driver.find_elements(By.CLASS_NAME, "_a9zm")
    len(blok_komentar)

    # membuat list kosong untuk menampung data
    list_komentar = []

    # looping untuk mengambil komentar dari setiap postingan
    for i in range(len(blok_komentar)):
        komentar = blok_komentar[i].get_attribute("textContent")
        list_komentar.append(komentar)

    # export ke json dengan nama kolom yang berbeda tiap postingan dan digabungkan kedalam 1 file
    df = pd.DataFrame(list_komentar, columns=['komentar pada post ke-'+str(post_list.index(post)+1)])
    df.to_json('komentar.json', orient='records', lines=True, mode='a')

    driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[2]").click()
    
    time.sleep(3)

driver.quit()